In [4]:
import os
from bs4 import BeautifulSoup
from jinja2 import Template

In [5]:
def camelcase(word: str) -> str:
    new_string = word.replace('-', ' ').split()
    if len(new_string) < 2:
        return word
    new_string = new_string[0].lower() + ''.join(word.capitalize() for word in new_string[1:])
    return new_string

In [6]:
def convert_html_to_jsx(element):
    tag_name = element.name
    if tag_name is None:
        return element.string

    attributes = []
    for attr, value in element.attrs.items():
        if attr in ["data-slot", "data-in-range", "data-orientation", "aria-label", "aria-labelledby", "aria-describedby", "aria-hidden", "aria-expanded"]:
            attr = attr
        elif attr == 'class':
            attr = 'className'
        elif attr == 'for':
            attr = 'htmlFor'
        else:
            attr = camelcase(attr)

        if attr == 'style':
            # Handling style
            temp = []
            for item in value.split(";"):
                if not item:
                    continue
                t1 = item.split(":")
                temp.append(":".join([camelcase(t1[0].strip()), f'"{t1[1].strip()}"']) if len(t1) > 1 else "beaf:'check'")
            new_value = "{{ %s }}" % (", ".join(temp))
            attributes.append(f'style={new_value}')
            continue
        attributes.append(f'{attr}="{" ".join(value) if isinstance(value, list) else value }"')

    attributes_str = ' '.join(attributes)

    if len(element.contents) == 0:
        return f'<{tag_name} {attributes_str} />'

    children = ''.join([convert_html_to_jsx(child) for child in element.contents])

    return f'<{tag_name} {attributes_str}>{children}</{tag_name}>'

In [7]:
def convert_html_file_to_react(html_file_path, react_file_path):
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')
    # body = soup.body

    # react_component = ''.join([convert_html_to_jsx(child) for child in body.children])
    react_component = ''.join([convert_html_to_jsx(soup)])

    component_template = Template('''import React from 'react';

const OutputComponent = () => (
                                {{ component_content | safe }}
                                );

export default OutputComponent;
''')

    rendered_component = component_template.render(component_content=react_component)
    # os.makedirs(os.path.dirname(react_file_path), exist_ok=True)
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(react_file_path), exist_ok=True)

    with open(react_file_path, 'w', encoding='utf-8') as file:
        file.write(rendered_component)

    print('React component has been created successfully!')

In [8]:
html_file_path = 'input.html'
react_file_path = 'output/OutputComponent.tsx'

In [9]:
convert_html_file_to_react(html_file_path, react_file_path)

React component has been created successfully!
